In [ ]:
'''
This notebook creates a maritime traffic network (MTN) from raw AIS data
- specify AIS data
- specify network hyperparameters

The MTM is generated in three steps:
- find significant turning points with Douglas Peucker algorithm
- cluster significant points to extract waypoints
- connect waypoints to get a graph
- optional: merge stop points that overlap and prune edges based on number of passages
'''

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from maritime_traffic_network import MaritimeTrafficNetwork
import folium
import warnings
import sys
import pickle
warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

In [ ]:
# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
print(sys.path)

# import modules
import visualize

In [ ]:
### Specify the AIS data to build the network from
datasize = 'full'          # part of the filename
location = 'stavanger'     # part of the filename
data_dates = ['202204']    # part of the filename

### Specify coordinate reference system
# 4326 for WGS 84 (global) // 32632 for UTM 32N (Stavanger, Oslo); 32634 for UTM 34N (Tromsø)
crs = 32632

### Specify network hyperparameters
tolerance = 10                      # Douglas Peucker tolerance parameter
method = 'HDBSCAN'                  # clustering algorithm: 'DBSCAN' , 'HDBSCAN', 'OPTICS'
metric = 'mahalanobis'              # distance metric: 'euclidean', 'mahalanobis', 'haversine'
min_samples = 10                    # clustering hyperparameter
min_cluster_size = min_samples      # clustering hyperparameter
eps = 0                             # clustering hyperparameter (DBSCAN only)
V = np.diag([1, 1, 0.01, 0.01, 1])  # mahalanobis distance parameter matrix (default: [1, 1, 0.01, 0.01, 1])
max_distance = 20                   # egde creation max_dist
max_angle = 45                      # edge creation max_angle                      
merge_stops = True                  # merging stop points that overlap, should always be True
merge_stops_speed = 2               # speed threshold for merging of stop points
pruning = 1                         # network pruning parameter (prunes edges with less than specified number of passages)

filenames = []  # placeholder for collection of all AIS input data files that the network was built from

In [ ]:
# read AIS data
for i in range(0, len(data_dates)):
    data_date = data_dates[i]
    # load processed AIS data from file
    filename = '../../data/processed/'+data_date+'_points_'+location+'_cleaned_meta_'+datasize+'_dualSplit_2.parquet'
    filenames.append(filename)
    gdf_part = gpd.read_parquet(filename)

    if i==0:
        gdf = gdf_part
    else:
        gdf = pd.concat([gdf, gdf_part])

data_date = str(data_dates)
filename = str(filenames)

In [ ]:
# filter the data according to ship type
#shiptypes = ['Tank', 'Last', 'Passasjer', None]
#gdf = gdf[gdf.skipsgruppe.isin(shiptypes)]

In [ ]:
# Transform to desired CRS
gdf.to_crs(crs, inplace=True)  # Transformation
gdf.head()

In [ ]:
# initialize maritime traffic network object
network = MaritimeTrafficNetwork(gdf, crs)
network.get_trajectories_info()

In [ ]:
# calculate significant turning points using Douglas Peucker algorithm
network.calc_significant_points_DP(tolerance)

In [ ]:
# detect waypoints using spatial clustering
network.calc_waypoints_clustering(method=method, min_samples=min_samples, min_cluster_size=min_cluster_size,
                                  eps=eps, metric=metric, V=V)

In [ ]:
# connect waypoints
network.make_graph_from_waypoints(max_distance=max_distance, max_angle=max_angle)

In [ ]:
# merge stop points
if merge_stops:
    network.merge_stop_points(max_speed=merge_stops_speed)
# prune network
network.prune_graph(pruning)

In [ ]:
# save hyperparameters
params = {
    'Data':filename,
    'DP_tolerance':tolerance,
    'clustering_method':method,
    'clustering_metric':metric,
    'clustering_min_samples':min_samples,
    'clustering_min_cluster_size':min_cluster_size,
    'clustering_eps':eps,
    'clustering_metric_V':V,
    'graph_generation_max_distance':max_distance,
    'graph_generation_max_angle':max_angle,
    'merge_stops_speed':merge_stops_speed
}
network.set_hyperparameters(params)

In [ ]:
# save network as pickle object
fileObj = open('../../data/interim/'+data_date+'_waypoints_DP'+str(tolerance)+'_'+method+str(min_samples)+'_'+location+'_'+datasize+'_UTM.obj', 'wb')
pickle.dump(network, fileObj)
fileObj.close()